In [ ]:
import pandas as pd
import warnings
import torch
import torchvision.models as models
import requests
import numpy as np
import time

from PIL import Image
from io import BytesIO
warnings.filterwarnings("ignore")
    

mobilenet = models.mobilenet_v2(pretrained=True)
mobilenet.cuda()
mobilenet.eval()

from torchvision import transforms
tr = transforms.Compose([            
 transforms.Resize(224),                   
 transforms.ToTensor(),                     
 transforms.Normalize(                      
 mean=[0.485, 0.456, 0.406],                
 std=[0.229, 0.224, 0.225]                  
 )])

a = make_embedding(images[images['item_id'] == 15969262].link.values)

def upload_transform(url):

    try:
        return torch.unsqueeze(tr(Image.open(BytesIO(requests.get(url).content))), 0).to('cuda')
    except Exception:
        return 0
    
def make_embedding(urls):
    
    embedding = 0
    k = 0
    control = 0
    for url in urls:
       
        try: 
            embedding += mobilenet(upload_transform(url))
            control += 1
        except Exception as e:
            print(e)
            k += 1
        
        if control >= 20:
            break
    try:   
        embedding /= control
    except Exception as e:
        print('FATAL ERROR')
        print('-------------------')
        print(e)
        print('-------------------')
        return a   # the value that i will fiind and delete
    return embedding.cpu().detach().numpy()[0]

In [ ]:
with open('items.csv') as f:
    item_ids = f.read()

item_ids = list(map(int, item_ids.split('\n')[1:-1]))

with open('imagenet_classes.txt') as f:
    labels = [line.strip() for line in f.readlines()]

df = pd.DataFrame(columns=labels)
df.index.name = 'item_id'
images = pd.read_csv('images.csv')



In [ ]:
n = 0
l = len(item_ids)

for item_id in item_ids[1282:]:
    n += 1
    print(f'{n}/{l} --- {item_id}' )
    links = images[images['item_id'] == item_id].link.values
    
    temp = make_embedding(links).cpu().detach().numpy()
    df.loc[item_id] = temp
    if n % 100 == 0:
        df.to_csv('embedding.csv')
        print('saved')

df.to_csv('embedding.csv')

In [ ]:
ker = df[(df['tench, Tinca tinca'] < a[0] + 0.0001) & (df['tench, Tinca tinca'] > a[0] - 0.0001)]['tench, Tinca tinca'].values[0]

df[df['tench, Tinca tinca'] == ker].index

In [ ]:
# delete "a" variables

df.drop([13672463, 13772806, 14084427, 14149728,  8783647,
            13507156, 13417356, 13641081, 13934101, 14158016, 13993003,
            13639584, 13924240, 13654052, 13517063, 13678700, 14146912,
            13987670, 13718072, 14084957, 14132470, 14106946, 13984281,
            14064382, 12994025, 13537658, 13934602, 12623113,  8614625,
            14108705, 14139940, 14108774, 14108926,  7914087, 12341531,
            14107316, 14137682], inplace=True)

In [ ]:
df.to_csv('embedding.csv')